# Assignment \#3: Importing Data (Part 2)
In this Assignment you will be practising alternative ways of sourcing data.

## Web scraping
For this Assignment you are tasked with scraping the following website: 'premierleague.com'

On this website you are looking for the results of football matches that took place in August 2019.

The results of these matches can be found by going to the url 'premierleague.com/match/\[match_ID\]' where match_ID will start at 46605 up to 46644. 

## Examing the code below.
The cell below contains limited capability of scraping the website for the results of a match and requires further functionality.

The code retrieves the html page containing the match results.
Finish the code to retrieve the following <div\> in which both team names as well as the score line for a match can be found:

Find and print the contents of <div class="teamsContainer"\></div\>


In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://premierleague.com/match/46605'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# insert code here
for contents in soup.find_all('div', {'class':'teamsContainer'}):
    print(contents.prettify())

<div aria-atomic="true" aria-live="polite" class="teamsContainer">
 <div class="team home">
  <a aria-hidden="true" class="badgeContainer" href="/clubs/10/Liverpool/overview">
   <span class="badge badge--large badge-image-container" data-size="50" data-widget="club-badge-image">
    <img class="badge-image badge-image--50" src="https://resources.premierleague.com/premierleague/badges/50/t14.png" srcset="https://resources.premierleague.com/premierleague/badges/50/t14.png, https://resources.premierleague.com/premierleague/badges/50/t14@x2.png 2x">
    </img>
   </span>
   <span class="badge badge--small badge-image-container" data-size="25" data-widget="club-badge-image">
    <img class="badge-image badge-image--25" src="https://resources.premierleague.com/premierleague/badges/25/t14.png" srcset="https://resources.premierleague.com/premierleague/badges/25/t14.png, https://resources.premierleague.com/premierleague/badges/25/t14@x2.png 2x">
    </img>
   </span>
  </a>
  <a aria-hidden="t

## Write code to clean the html code stored in 'teamsContainer' to output the team names and scoreline of the given match.

In [11]:
# write code here
teamContainer = soup.find('div', attrs = {'class':'teamsContainer'})

teamnames = teamContainer.find_all('a', attrs = {'class': 'teamName'})
scoreline = teamContainer.find_all('div', attrs = {'class': 'score fullTime'})
teamNames =[]
score = []

for x in teamnames:
    teamNames.append(x.find('span', attrs = {'class':'long'}).text)
for x in scoreline:
    score.append(x.text)
print(teamNames[0] + ' vs ' + teamNames[1] + ' = ' + str(score))

Liverpool vs Norwich City = ['4-1']


## Now we want to retrieve the team names as well of the scoreline for all the matches in August (match ID: 46605 to 46643).
Save the names of the home team, away team, the number of goals scored by the home team and the number of goals scored by the away team in a dataframe.

Afterwards print out the dataframe.

Note: If correctly implemented, it should take up to a 3 minutes to scrape these results.

In [35]:
# write code here
df = pd.DataFrame()
HomeTeam = []
AwayTeam = []
ScoresH =[]
ScoresA = []
link = "https://premierleague.com/match/"

for id in range(46605,46644):
    url = link + str(id)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    teamContainer = soup.find('div', attrs = {'class':'teamsContainer'})
    teamHome = teamContainer.find('div', attrs = {'class': 'team home'})
    teamAway = teamContainer.find('div', attrs = {'class': 'team away'})
    scoreline = teamContainer.find('div', attrs = {'class': 'score fullTime'})
    HomeTeam.append(teamHome.find('span', attrs = {'class':'long'}).text)
    AwayTeam.append(teamAway.find('span', attrs = {'class', 'long'}).text)
    ScoresH.append(scoreline.text.split('-')[0])
    ScoresA.append(scoreline.text.split('-')[1])
df = pd.DataFrame({"Home Team":HomeTeam, "Away Team":AwayTeam, "Home Team Goals":ScoresH, "Away Team Goals":ScoresA})
print(df)

                   Home Team                 Away Team Home Team Goals  \
0                  Liverpool              Norwich City               4   
1                Bournemouth          Sheffield United               1   
2                    Burnley               Southampton               3   
3             Crystal Palace                   Everton               0   
4             Leicester City   Wolverhampton Wanderers               0   
5          Tottenham Hotspur               Aston Villa               3   
6                    Watford  Brighton and Hove Albion               0   
7            West Ham United           Manchester City               0   
8          Manchester United                   Chelsea               4   
9           Newcastle United                   Arsenal               0   
10                   Arsenal                   Burnley               2   
11               Aston Villa               Bournemouth               1   
12  Brighton and Hove Albion          

## Print a list of all unique teams.

In [40]:
# write code here
home = df["Home Team"].unique()
print(sorted(home))

['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton and Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester United', 'Newcastle United', 'Norwich City', 'Sheffield United', 'Southampton', 'Tottenham Hotspur', 'Watford', 'West Ham United', 'Wolverhampton Wanderers']


## Using the scores of each match, compute the current standings.
Use the following rules to calculate the points for a given team:

For a win, award a team 3, points.

A draw earns both teams 1 point.

No points are earned for a loss.

### Print the resulting dataframe ordered by total points.
Your dataframe should have the following columns: Team Name, Total points.


In [44]:
# write code here
import numpy as np
TeamName = list(df['Away Team'].unique())
points = np.zeros(len(TeamName))

for x in df.values:
    if x[2] > x[3]:
        points[TeamName.index(x[0])] = points[TeamName.index(x[0])] + 3
    elif x[2] < x[3]:
        points[TeamName.index(x[1])] = points[TeamName.index(x[1])] + 3
    else:
        points[TeamName.index(x[0])] = points[TeamName.index(x[0])] + 1
        points[TeamName.index(x[1])] = points[TeamName.index(x[1])] + 1
points_df = pd.DataFrame({'Team Name': TeamName, 'Points': points})
sorted_points_df = points_df.sort_values('Points', ascending = False)
print(sorted_points_df)

                   Team Name  Points
18                 Liverpool    12.0
7            Manchester City    10.0
13            Leicester City     8.0
3                    Everton     7.0
17            Crystal Palace     7.0
9                    Arsenal     7.0
19         Manchester United     5.0
15         Tottenham Hotspur     5.0
8                    Chelsea     5.0
1           Sheffield United     5.0
12           West Ham United     4.0
16          Newcastle United     4.0
10                   Burnley     4.0
11               Bournemouth     4.0
6   Brighton and Hove Albion     4.0
2                Southampton     4.0
5                Aston Villa     3.0
4    Wolverhampton Wanderers     3.0
0               Norwich City     3.0
14                   Watford     1.0
